In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mh-nmh/ww-binary-dataset/dataset_dict.json
/kaggle/input/mh-nmh/ww-binary-dataset/validation/state.json
/kaggle/input/mh-nmh/ww-binary-dataset/validation/dataset_info.json
/kaggle/input/mh-nmh/ww-binary-dataset/validation/data-00000-of-00001.arrow
/kaggle/input/mh-nmh/ww-binary-dataset/test/state.json
/kaggle/input/mh-nmh/ww-binary-dataset/test/dataset_info.json
/kaggle/input/mh-nmh/ww-binary-dataset/test/data-00000-of-00001.arrow
/kaggle/input/mh-nmh/ww-binary-dataset/train/state.json
/kaggle/input/mh-nmh/ww-binary-dataset/train/dataset_info.json
/kaggle/input/mh-nmh/ww-binary-dataset/train/data-00000-of-00001.arrow
/kaggle/input/model-weights/roberta_model.pt


In [2]:
!cp -r /kaggle/input/mh-nmh/ww-binary-dataset ./

In [3]:
!pip install transformers datasets

In [4]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
# import torchtext.functional as F

from transformers import AutoTokenizer, XLMRobertaForSequenceClassification,DataCollatorWithPadding

from IPython.display import Markdown
import pandas as pd

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
# dataset = DatasetDict.load_from_disk('ww-binary-dataset-small')
dataset = DatasetDict.load_from_disk('/kaggle/working/ww-binary-dataset')

In [6]:
train_datapipe = dataset['train']
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base",local_files_only=False)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
train_datapipe = train_datapipe.map(lambda x:  tokenizer(x['Text'],truncation=True,padding=True))
train_datapipe.set_format("torch")
train_datapipe = train_datapipe.remove_columns(["Text"])

  0%|          | 0/460000 [00:00<?, ?ex/s]

In [8]:
train_dataloader = DataLoader(
    train_datapipe, batch_size=16, 
    collate_fn=data_collator
)

In [9]:
def on_train_end_manual(model):
        # Compute accuracy on the validation set
        # val_loader = self.trainer.val_dataloader()
        res = []
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for i,batch in enumerate(tqdm(train_dataloader)):
                input = batch["input_ids"].clone().detach().to(DEVICE)
                output = model(input)
                logits = output.logits
                preds = torch.argmax(logits, axis=1)
                probs = torch.softmax(logits, dim=1)
                correct += torch.sum(preds == batch["labels"].clone().detach().to(DEVICE)).item()
                total += len(preds)
                labels = batch['labels']
                targets = probs[:,1].tolist()
                res.extend(np.array([labels.numpy(), targets]).T.tolist())
#                 print(total)
        val_acc = correct / total

        print(f"Final validation accuracy: {val_acc}")
        return res

In [10]:
# from transformers import XLMRobertaConfig
num_classes = 2

# configuration = XLMRobertaConfig("xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base",num_labels=num_classes, from_tf=False)
model.load_state_dict(torch.load('/kaggle/input/model-weights/roberta_model.pt'))
model.to(DEVICE); 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [11]:
result = on_train_end_manual(model)

100%|██████████| 28750/28750 [2:05:12<00:00,  3.83it/s]

Final validation accuracy: 0.9805456521739131


In [12]:
np.save('train_inf',result)